# PyTorch Computer Vision

## 0. Computer vision libraries in PyTorch
* torchvision - base library for computer vision
* torchvision.da - get datasets and data loading functions
* torchvision.models - pretrained computer vision models that can be leveraged for your models
* torchvision.transform - functions for manupulating your vision data to make it suitable to use
* torch.utils.data.Dataset - Base dataset class for pytorch
* torch.utils.DataLoader - Creates a Python iterable over a dataset

In [ ]:
import torch
from torch import nn

# import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

: 

## Getting a dataset

The dataset we are going to use Fashion-MNIST from torchviosion.datasets

In [ ]:
# Setup Training data
train_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download = True,
    transform = ToTensor(),        # Transforms image data into tensors
    target_transform=None,
)

test_data = datasets.FashionMNIST(
    root = "data",
    train=False,
    download = True,
    transform = ToTensor(),        # Transforms image data into tensors
    target_transform=None,
)

In [ ]:
len(train_data), len(test_data)

In [ ]:
# See the first training example
image_info = train_data
image_info

In [ ]:
image,label = train_data[0]
print(image,"\n")
print(label)

In [ ]:
class_names=train_data.classes    # So we have 9 classes in our data sets
class_names

In [ ]:
class_to_idx = train_data.class_to_idx      # index of all the classes
class_to_idx

In [ ]:
# Accessing the image data (tensor) within the tuple using indexing.
image_data = image[0]  # Accessing the first item which is actual image data
image_data.shape       # Access the shape of the image data stored as a Tensor.
# shape is in oreder C,H,W there is only one colour channel as the data is in grey scale

In [ ]:
# The labels/outputs should be an integer from 0-9
train_data.targets

In [ ]:
image, label = train_data[0]
plt.imshow(image.squeeze())
plt.title(label)
plt.axis('off')  # this hides the axis

In [ ]:
plt.imshow(image.squeeze(), cmap="gray")    # Grey scale image of the data
plt.title(class_names[label])


In [ ]:
# Plot random images
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols = 4, 4
for i in range (1, rows*cols+1, 1):
  random_idx = torch.randint(0, len(train_data), (1,)).item()
  print (random_idx)
  img, label = train_data[random_idx]
  fig.add_subplot(rows, cols, i)
  plt.imshow(img.squeeze(), cmap="gray")
  plt.title(class_names[label])
  plt.axis('off')  # this hides the axis

These items of clothing connot be distinguished using pure linear model so we will need non linearities

## 2 Prepare DataLoader
Right now, our data is in the form of PyTorch Datasets.

DataLoader turn our dataset into a python iterable

More specifically we convert our data into small batches
why we do this?
* it is more efficient to look at a small batach at one time for the computer
* It gives our nn more chances to update its gradients per epoch

In [ ]:
from torch.utils.data import DataLoader

#setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into batches
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)             # To give diversity in each batch
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)
train_dataloader,test_dataloader

In [ ]:
print(f"length of data/no batch size : {len(train_dataloader)}")         # there are 1875 batches of 32 images

In [ ]:
# Check out whats inside the training dataloader
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
# Visualizing a batch
import torch
import matplotlib.pyplot as plt

torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), (1,)).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]

# Plotting
plt.figure(figsize=(4, 4))
plt.imshow(img.squeeze(), cmap="gray")
plt.title(class_names[label])
plt.axis('off')  # this hides the axis

we have now successfully got our data ready

## 3 Model 0 (Baseline model)

When building and experimenting on ML models , start with a baseline model

baseline model: simple model and add complexity as we go so like iterative enhancement model

In [ ]:
# Creating a flatten layer
flatten_model = nn.Flatten()

x = train_features_batch[0]

output = flatten_model(x)         # Perform forward pass

print(x.shape, output.shape)    # converts multi-dimensional tensor into a stright line or flattens it

# we need to flatten our data to create a layer

In [ ]:
from torch import nn
class fashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_shapes:int,
               hidden_units:int,
               output_shapes:int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shapes,
                  out_features=hidden_units),
        nn.Linear(in_features=hidden_units,
                  out_features=output_shapes)

    )

  def forward(self, x):
    return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)

model_0 = fashionMNISTModelV0(
    input_shapes = 784,
    hidden_units = 10,
    output_shapes=len(class_names)
).to("cpu")

model_0

In [ ]:
model_0.state_dict()

### 3.1 Loss function and optimizer and evaluation metrics
* Loss function - since we are working with MCC data , we use cross entropy
* optimizer - SGD
* evaluation metric - accuracy

In [ ]:
# importing helper functions
import requests
from pathlib import Path
if Path("helper_function.py").is_file():
  print("already downloaded")
else:
  print("downloading ")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
# we have accuracy function in the helper function file
from helper_functions import accuracy_fn

In [ ]:
accuracy_fn

In [ ]:
# Loss function and optimizeer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.001)


### 3.3 Creating a function to time our experiment
to check the models efficiency

In [ ]:
from timeit import default_timer as timer
def print_train_time(start: float,
                     end:float,
                     device: torch.device =None):
  """ Prints differnce between between start and end time"""
  total_time = end - start
  print(f"Train time on {device} : {total_time:.3f} seconds")
  return total_time


In [ ]:
start_time = timer()
# some code...
end_time = timer()
print_train_time(start=start_time, end=end_time, device="cpu")


### 3.3 Creating a training loop and training a model on batches of data

1. Loop through epochs
2. Loop through batches , perform Training steps, calculate the train loss *per batch*
3.Loop through testing batches, perform testing steps, calculate the test loss per batch
4. print out whats happening


In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# Set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_cpu = timer()

# Set the number of epochs (we'll keep this small for faster training times)
epochs = 3

# Create training and testing loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    ### Training
    train_loss = 0
    # Add a loop to loop through training batches
    for batch, (X, y) in enumerate(train_dataloader):
        model_0.train()
        # 1. Forward pass
        y_pred = model_0(X)

        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred, y)
        train_loss += loss # accumulatively add up the loss per epoch

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Print out how many samples have been seen
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    # Divide total train loss by length of train dataloader (average loss per batch per epoch)
    train_loss /= len(train_dataloader)

    ### Testing
    # Setup variables for accumulatively adding up loss and accuracy
    test_loss, test_acc = 0, 0
    model_0.eval()
    with torch.inference_mode():
        for X, y in test_dataloader:
            # 1. Forward pass
            test_pred = model_0(X)

            # 2. Calculate loss (accumulatively)
            test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

            # 3. Calculate accuracy (preds need to be same as y_true)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_acc /= len(test_dataloader)

    ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")

# Calculate training time
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,
                                           end=train_time_end_on_cpu,
                                           device=str(next(model_0.parameters()).device))